### .txt to .json

In [9]:
using System.IO;
using System.Text.Json;
using System.Text.Encodings.Web;
using System.Text.Unicode;

var filePath = @"C:\Users\Faruque\source\repos\quranpwa\quranpwa\src\assets\quran-texts\quran-uthmani-min.txt";
var textLines = File.ReadAllLines(filePath).Take(6236);

var options = new JsonSerializerOptions
{
    Encoder = JavaScriptEncoder.Create(UnicodeRanges.All),
    WriteIndented = true
};

var ayatArrayJson = JsonSerializer.Serialize(textLines, options);
 
File.WriteAllText(filePath + ".json", ayatArrayJson);


## Extract data from sqlite database and transform

In [1]:
#r "nuget: System.Data.SQLite, *-*"
#r "nuget: Microsoft.DotNet.Interactive.ExtensionLab, *-*"
#r "nuget: Dapper, *-*"

using System.IO;
using System.Text.Json;
using System.Text.Encodings.Web;
using System.Text.Unicode;
using System.Data.SQLite;
using Dapper;

var jsonSerializerOptions = new JsonSerializerOptions
{
    Encoder = JavaScriptEncoder.Create(UnicodeRanges.All),
    WriteIndented = true
};

Installed Packages Dapper, 2.1.44 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.24229.4 System.Data.SQLite, 1.0.118

Loading extension script from `C:\Users\Faruque\.nuget\packages\microsoft.dotnet.interactive.extensionlab\1.0.0-beta.24229.4\interactive-extensions\dotnet\extension.dib`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

ExplainCode 
 This extension generates Sequence diagrams from csharp code using Mermaid kernel.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay an interactive session via automation.

Loading extensions from `C:\Users\Faruque\.nuget\packages\microsoft.data.analysis\0.21.0\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
var dbPath = @"D:\DatabaseBackups\QuranDB\";

bool SaveAsJson(string dbFilename, string saveToFolder){
    var dbFileFullName = dbPath + dbFilename + ".db";
    
    if (!File.Exists(dbFileFullName)) 
        return false;

    using (var connenction = new SQLiteConnection($"Data Source={dbFileFullName};Mode=Memory;Cache=Shared"))
    {
        var result = connenction.Query("SELECT * FROM verses ORDER BY sura, ayah");

        var ayatTexts = result.Select(s=> s.text as string);
        var ayatTextsJson = JsonSerializer.Serialize(ayatTexts, jsonSerializerOptions);
    
        File.WriteAllText(dbPath + "extractedJson\\" + saveToFolder + "\\" + dbFilename.Replace('_','-') + ".json", ayatTextsJson);
    }

    return true;
}

//SaveAsJson("quran");


In [3]:
using System.Globalization;

string GetLocaleByLanguageCode(string twoLetterISOLanguageName)
{
    var cultures = CultureInfo.GetCultures(CultureTypes.AllCultures & ~CultureTypes.NeutralCultures);
    return cultures.FirstOrDefault(f=>f.TwoLetterISOLanguageName == twoLetterISOLanguageName)?.Name;
}

//Console.WriteLine(GetLocaleByLanguageCode("bn"));

In [4]:
var translationListJson = File.ReadAllText(dbPath + "translation_list_all.json");

record TranslationItem(string Name, string lastModified, int downloadType, string fileName, bool isZip, int id, string language, string translator);

var translationItems = JsonSerializer.Deserialize<TranslationItem[]>(translationListJson);

var extractedTranslationItems = new List<TranslationItem>();

foreach (var translationItem in translationItems){
    var folderName = translationItem.downloadType == 1 ? "tafsirs" : "translations";
    if (SaveAsJson(translationItem.fileName, folderName))
        extractedTranslationItems.Add(translationItem);
}

IEnumerable MapTranslationObject(IEnumerable<TranslationItem> pTranslations){
    return pTranslations.OrderBy(o=>o.Name).Select(item => new { 
        id = item.fileName.Replace('_','-'),
        name = item.Name.Split(" - ")[1],
        languageName = item.Name.Split(" - ")[0],
        translator = item.translator,
        language = item.language,
        locale = GetLocaleByLanguageCode(item.language)
    });
}

var extractedTranslationList = MapTranslationObject(extractedTranslationItems.Where(f=>f.downloadType != 1));
var extractedTranslationListJson = JsonSerializer.Serialize(extractedTranslationList, jsonSerializerOptions);
File.WriteAllText(dbPath + "extractedJson\\translation-list.json", extractedTranslationListJson);

var extractedTafsirList = MapTranslationObject(extractedTranslationItems.Where(f=>f.downloadType == 1));
var extractedTafsirListJson = JsonSerializer.Serialize(extractedTafsirList, jsonSerializerOptions);
File.WriteAllText(dbPath + "extractedJson\\tafsir-list.json", extractedTafsirListJson);


Word by word translations

In [5]:
var wordDbPath = @"D:\DatabaseBackups\QuranDB\corpus\";

bool SaveWbwTranslationAsCsv(string dbFilename){
    var dbFileFullName = wordDbPath + dbFilename + ".db";
    
    if (!File.Exists(dbFileFullName)) 
        return false;

    using (var connenction = new SQLiteConnection($"Data Source={dbFileFullName};Mode=Memory;Cache=Shared"))
    {
        var result = connenction.Query("SELECT * FROM quran");

        var words = result.Select(s=> s.tr as string);
    
        File.WriteAllText(wordDbPath + "extractedCsv\\" + dbFilename.Replace('_','-') + ".txt",string.Join(Environment.NewLine, words));
    }

    return true;
}

//SavewbwTranslationAsCsv("words_bn");

In [6]:
var wbwTranslationListJson = File.ReadAllText(wordDbPath + "wbw_db_list.json");

record WbwTranslationItem(string name, string language, string lastModified);

var wbwTranslationItems = JsonSerializer.Deserialize<WbwTranslationItem[]>(wbwTranslationListJson);

var extractedWbwTranslationItems = new List<WbwTranslationItem>();

foreach (var wbwTranslationItem in wbwTranslationItems){
    if (SaveWbwTranslationAsCsv("words_" + wbwTranslationItem.language))
        extractedWbwTranslationItems.Add(wbwTranslationItem);
}

IEnumerable MapTranslationObject(IEnumerable<WbwTranslationItem> pTranslations){
    return pTranslations.OrderBy(o=>o.name).Select(item => new { 
        id = "words-" + item.language,
        name = item.name,
        language = item.language,
    });
}

var extractedWbwTranslationList = MapTranslationObject(extractedWbwTranslationItems);
var extractedWbwTranslationListJson = JsonSerializer.Serialize(extractedWbwTranslationList, jsonSerializerOptions);
File.WriteAllText(wordDbPath + "extractedCsv\\wbw-translation-list.json", extractedWbwTranslationListJson);
